In [ ]:
#import the required libraries
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import *
!pip install psycopg2
import psycopg2
import sqlalchemy
import boto3
import numpy as np

In [ ]:
#declaring the credentials 
database = {'database-1':{
    'Name':'Cars',
    'User': 'postgres',
    'Password':'Forget123',
    'Host':'database-1.ctibgerv5wfi.us-east-1.rds.amazonaws.com',
    'Port':5432,
},
           }

In [ ]:
db = database['database-1']

In [ ]:
engine_string = "postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}".format(
    user=db['User'],
    password=db['Password'],
    host = db['Host'],
    port = db['Port'],
    database = db['Name'],)

In [ ]:
#creating the engine
engine = create_engine(engine_string)

In [ ]:
#reading the table as pandas df
cars_df = pd.read_sql_table('cars',engine)

In [ ]:
cars_df 

In [ ]:
sales_df = pd.read_sql_table('cars_sales',engine)

In [ ]:
sales_df

In [ ]:
customer_df = pd.read_sql_table('cars_customers',engine)

In [ ]:
customer_df

In [ ]:
policy_df = pd.read_sql_table('cars_policy',engine)

In [ ]:
policy_df 

In [ ]:
#importing table data from dynamodb
db_resource = boto3.resource('dynamodb',region_name = 'us-east-1')
claims_table = db_resource.Table('Claims')
response = claims_table.scan()
claims_df = pd.json_normalize(response['Items'])

claims_df.head()

In [ ]:
claims_df

In [ ]:
#convering the sold_on to datetime
sales_df['month'] = pd.to_datetime(sales_df['sold_on']).dt.strftime('%Y-%m')

In [ ]:
#grouping the data based on month
sales_by_month = sales_df.groupby(['month']).sum()

In [ ]:
sales_by_month

In [ ]:
metric1 = sales_by_month

In [ ]:
#dropping the columns and the null values data
metric1 = metric1.drop(['sales_id','car_id'],axis=1)
metric1 = metric1.drop(['1900-01'],axis=0)

In [ ]:
metric1 = metric1.drop(['1900-01'],axis=0)

In [ ]:

metric1

In [ ]:
metric1.rename(columns={'selling_price':'Total_sales_amount'},inplace=True)

In [ ]:
metric1

In [ ]:
#creating metric2 on total cars sold
total_cars=sales_df.groupby(['month'])['car_id'].agg('count').reset_index().rename(columns={'car_id':'Total_Cars_Sold'})

In [ ]:
metric1=metric1.merge(total_cars[['month','Total_Cars_Sold']],left_on=['month'],right_on=['month'],how='left')

In [ ]:
metric1

In [ ]:
claims = claims_df

In [ ]:
#converting null into Nan for converting column to datetime# and datatype to datetime
claims['Claim_Processed_On']=claims['Claim_Processed_On'].replace('null',np.NaN)
claims['Claim_Processed_On']=pd.to_datetime(claims['Claim_Processed_On'],dayfirst=True)

In [ ]:
claims['month'] = pd.to_datetime(claims['Claim_Processed_On']).dt.strftime('%Y-%m')
claims

In [ ]:
#calculating the sum of claims to get claims amount
claims['claim amount']=(pd.to_numeric(claims['property'],errors='coerce'))+(pd.to_numeric(claims['injury'],errors='coerce'))+(pd.to_numeric(claims['vehicle'],errors='coerce'))

In [ ]:
#groupping data according to month
vok=claims.groupby(['month']).sum()

In [ ]:
vok

In [ ]:
#changing datatype to datetime
claims['Claim_Logged_On']=claims['Claim_Logged_On'].replace('null',np.NaN)
claims['Claim_Logged_On']=pd.to_datetime(claims['Claim_Logged_On'],dayfirst=True)
claims['processing days']=(claims['Claim_Processed_On']-claims['Claim_Logged_On']).dt.days

In [ ]:
#groupping data according to month
claims_metric=claims.groupby(['month']).mean()

In [ ]:
claims_metric

In [ ]:
#transfering metrics to s3
from io import StringIO
bucket = 'carsdataimagineerok'
csv = StringIO()
metric1.to_csv(csv, index = False)
s3_resource = boto3.resource('s3')
s3_resource.Object(bucket,'sales_metric.csv').put(Body=csv.getvalue())

In [ ]:
#transfering metrics to s3
from io import StringIO
bucket = 'carsdataimagineerok'
csv = StringIO()
claims_metric.to_csv(csv)
s3_resource = boto3.resource('s3')
s3_resource.Object(bucket,'claims_metric.csv').put(Body=csv.getvalue())